# Standalone reading functions for nanonis files

In [5]:
def read_nanonis_experiment(filename):    
    # Reads the type of experiment from the header of any nanonis file. Examples:
    # - 'Experiment' for freq. sweep files
    # - 'bias spectroscopy'
    # - 'Z spectroscopy'
    
    with open(filename, "rb") as data_file:
        dummy_file = data_file.read()

        position = dummy_file.find(b'Experiment')
        data_file.seek(position)
        #data_type = data_file.readline().decode('ascii').split()[1:]
        data_type = data_file.readline().decode('ascii')[11:]
    return data_type

def read_nanonis_fsweep_header(filename):
    # Read the header of a fsweep file into a python dictionary
    # f_res (Hz), Phase (deg), Amp/Exc (nm/mV)
    
    header = {} # define empty dictionary
    with open(filename, "rb") as data_file:
        dummy_file = data_file.read()

        position = dummy_file.find(b'Experiment')
        data_file.seek(position)
        line = data_file.readline().decode('ascii').split()
        header[line[0].lower()] = 'freq-sweep'
             
        position = dummy_file.find(b'Date')
        data_file.seek(position)
        line = data_file.readline().decode('ascii').split()
        header['date'] = line[1]
        header['time'] = line[2]

        position = dummy_file.find(b'f_res')
        data_file.seek(position)
        line = data_file.readline().decode('ascii').split()
        header['f-res-hz'] = float(line[-1])

        position = dummy_file.find(b'Q')
        data_file.seek(position)
        line = data_file.readline().decode('ascii').split()
        header['q-factor'] = int(line[1])

        position = dummy_file.find(b'Phase')
        data_file.seek(position)
        line = data_file.readline().decode('ascii').split()
        header['phase-deg'] = float(line[-1])

        position = dummy_file.find(b'Amp/Exc')
        data_file.seek(position)
        line = data_file.readline().decode('ascii').split()
        header['amp-exc-nm-mv'] = float(line[-1])

        position = dummy_file.find(b'Number of points')
        data_file.seek(position)
        line = data_file.readline().decode('ascii').split()
        header['points'] = int(line[-1]) 
        return(header)

def read_nanonis_curve(filename):
    # Read the numeric data from ASCII nanonis file
    import pandas as pd
    with open(filename, "rb") as data_file:
        dummy_file = data_file.read()

        position = dummy_file.find(b'[DATA]')
        data_file.seek(position)
        data_file.readline()
        data_table = data_file.read().decode('ascii')
    return pd.read_csv(StringIO(data_table), sep='\t')